In [ ]:
!pip install chromadb -q
!pip install sentence-transformers -q

In [ ]:
#imports
import chromadb
import pandas as pd
from itertools import chain

In [ ]:
client = chromadb.Client() # default in memory
collection = client.create_collection('quotes')

In [ ]:
#client.delete_collection('quotes')

In [ ]:
df = pd.read_csv('sample_data/quotes.csv')
df.head(5)

In [ ]:
# add quotes as documents and person, topic as metadata collection itmes
# in-build embeddings
collection.add(
    ids=[str(i) for i in range(0, len(df))],
    documents=df['quote'].to_list(),
    metadatas=[{'person': p, 'topic': t} for p,t in zip(df['person'].to_list(), df['topic'].to_list())],
)


In [ ]:
# show document and mapped embedding
collection.get(ids=['1'],include=['documents','embeddings'])


In [ ]:
my_query = 'what is love'
results = collection.query(
    query_texts=my_query,
    n_results=3)


In [ ]:
for doc in results['documents'][0]:
  print(doc)

Love is not only something you feel, it is something you do.
Love is an irresistible desire to be irresistibly desired.
Where there is love there is life.


In [ ]:
my_query = 'what is green'
results = collection.query(
    query_texts=my_query,
    n_results=3)
for doc in results['documents'][0]:
  print(doc)

Liberty, when it begins to take root, is a plant of rapid growth.
Freedom is nothing else but a chance to be better.
Freedom is nothing but a chance to be better.


In [ ]:
my_query = 'what is red'
results = collection.query(
    query_texts=my_query,
    n_results=3)
for doc in results['documents'][0]:
  print(doc)

I think, therefore I am.
Fear is a reaction. Courage is a decision.
Creativity is seeing what others see and thinking what no one else ever thought.


In [ ]:
my_query = 'what to do when you are sad'
results = collection.query(
    query_texts=my_query,
    n_results=3)
for doc in results['documents'][0]:
  print(doc)

Question everything. Learn something. Answer nothing.
Do not pray for easy lives. Pray to be stronger men.
The purpose of our lives is to be happy.


In [ ]:
collection.count()

100

In [ ]:
print(collection)

name='quotes' id=UUID('745f9a45-cf56-41f8-9653-aabdf43cd7e0') metadata=None tenant='default_tenant' database='default_database'


## Change similarity function and use filters
Changing similarity distance while creating collection. Default is L2
https://docs.trychroma.com/usage-guide#changing-the-distance-function

In [ ]:
client.delete_collection('quotes')

In [ ]:
collection = client.create_collection(
        name="quotes",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

collection.add(
    ids=[str(i) for i in range(0, len(df))],
    documents=df['quote'].to_list(),
    metadatas=[{'person': p, 'topic': t} for p,t in zip(df['person'].to_list(), df['topic'].to_list())],
)

In [ ]:
my_query = 'what is love'
results = collection.query(
    query_texts=my_query,
    n_results=3)
for doc in results['documents'][0]:
  print(doc)

Love is not only something you feel, it is something you do.
Love is an irresistible desire to be irresistibly desired.
Where there is love there is life.


In [ ]:
# use where to specify metadata
my_query = 'what is love'
results = collection.query(
    query_texts=my_query,
    n_results=3,
    #where={"topic": "Individuality"} # topic is equal to 'Individuality'
    where={"topic": {"$ne": "Love"}} # topic is not equal to 'Love'
    )
for doc in results['documents'][0]:
  print(doc)
print(results)

When the power of love overcomes the love of power, the world will know peace.
The purpose of our lives is to be happy.
Success is not the key to happiness. Happiness is the key to success. If you love what you are doing, you will be successful.
{'ids': [['71', '67', '17']], 'distances': [[0.6005175113677979, 0.6716969013214111, 0.745746374130249]], 'metadatas': [[{'person': 'Jimi Hendrix', 'topic': 'Peace'}, {'person': 'Dalai Lama', 'topic': 'Life'}, {'person': 'Albert Schweitzer', 'topic': 'Success'}]], 'embeddings': None, 'documents': [['When the power of love overcomes the love of power, the world will know peace.', 'The purpose of our lives is to be happy.', 'Success is not the key to happiness. Happiness is the key to success. If you love what you are doing, you will be successful.']], 'uris': None, 'data': None}
